# Example of computing vibrational energy levels of a triatomic molecule using the Taylor series expansion approximation for the kinetic and potential energy operators

Compute vibrational states of $\text{H}_2\text{S}$ molecule using the potential energy surface from
[A. A. A. Azzam, J. Tennyson, S. N. Yurchenko, O. V. Naumenko, "ExoMol molecular line lists - XVI. The rotation–vibration spectrum of hot H2S", MNRAS 460, 4063–4074 (2016)](https://doi.org/10.1093/mnras/stw1133)


In [1]:
from typing import Callable, List

import jax
import jax.numpy as jnp
import numpy as np
from numpy.polynomial.hermite import hermder, hermgauss, hermval
from scipy import optimize
from scipy.special import factorial

from vibrojet.keo import Gmat, com
from vibrojet.potentials import h2s_AYT2
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Compute equilibrium coordinates, around which the Taylor series expansions will be built

In [2]:
vmin = optimize.minimize(h2s_AYT2.poten, [1, 1, 90 * np.pi / 180])
r0 = vmin.x
v0 = vmin.fun
print("Equilibrium coordinates:", r0)
print("Min of the potential:", v0)

Equilibrium coordinates: [1.3358387  1.3358387  1.61042427]
Min of the potential: -0.0007846164047977397


Define mapping function from internal valence coordinates, $r_1$, $r_2$, and $\alpha$, to Cartesian coordinates

In [3]:
masses = [31.97207070, 1.00782505, 1.00782505]  # masses of S, H, H

ncoo = len(r0)


@com(masses)
def internal_to_cartesian(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Utility function for generating index combinations corresponding to products of primitive basis functions as well as derivative indices used in the expansion of operators

In [4]:
def generate_prod_ind(
    indices: List[List[int]],
    select: Callable[[List[int]], bool] = lambda _: True,
    batch_size: int = None,
):
    no_elem = np.array([len(elem) for elem in indices])
    tot_size = np.prod(no_elem)
    if batch_size is None:
        batch_size = tot_size
    no_batches = (tot_size + batch_size - 1) // batch_size

    for ibatch in range(no_batches):
        start_ind = ibatch * batch_size
        end_ind = np.minimum(start_ind + batch_size, tot_size)
        batch_ind = np.arange(start_ind, end_ind)
        multi_ind = np.array(np.unravel_index(batch_ind, no_elem))
        indices_out = np.array(
            [indices[icoo][multi_ind[icoo, :]] for icoo in range(len(indices))]
        ).T
        select_ind = np.where(np.asarray([select(elem) for elem in indices_out]))
        yield indices_out[select_ind], multi_ind[:, select_ind[0]]

Compute Taylor series expansion of the $G$-matrix and potential

$$
G_{\lambda,\mu}=\sum_{\mathbf{t}} g_{\mathbf{t},\lambda,\mu} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

$$
V=\sum_{\mathbf{t}} f_{\mathbf{t}} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

 where the derivative multi-index $\mathbf{t}$ is stored in `deriv_ind`, and the expansion coefficients $g_{\mathbf{t},\lambda,\mu}$ and $f_{\mathbf{t}}$ in `Gmat_coefs[:len(deriv_ind),:3N, :3N]` and `poten_coefs[:len(deriv_ind)]` respectively, where $N$ is the number of atoms

In [ ]:
max_pow = 8
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind[:2]) <= max_pow)
)

print("Max expansion power:", max_pow)
print("Number of expansion terms:", len(deriv_ind))

print("compute expansion of potential ...")
poten_coefs = deriv_list(h2s_AYT2.poten, deriv_ind, r0, if_taylor=True)

print("compute expansion of G-matrix ...")
Gmat_coefs = deriv_list(
    lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, r0, if_taylor=True
)

Max expansion power: 8
Number of expansion terms: 405
compute expansion of potential ...


Define primitive Hermite basis functions $\mathcal{H}_n(x)$ and their derivatives $d\mathcal{H}_n(x)/dx$

In [ ]:
def hermite(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi)
    f = hermval(np.asarray(x), c) * np.exp(-(x**2) / 2)
    return f.T

def hermite_deriv(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi)
    h = hermval(np.asarray(x), c)
    dh = hermval(np.asarray(x), hermder(c, m=1))
    f = (dh - h * x) * np.exp(-(x**2) / 2)
    return f.T

Define mapping between the coordinate $x\in (-\infty, \infty)$ of Hermite basis functions and the internal valence coordinates as $r_1=a_1 x_1 + b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$. The parameters $a_1,b_1,...,b_3$ are determined by mapping the vibrational Hamiltonian in valence coordinates onto the harmonic-oscillator Hamiltonian.

In [ ]:
mask = deriv_ind != 0

# Elements of G-matrix at equilibrium
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(Gmat_coefs[ind0])[:ncoo]

# Second-order derivative of potential at equilibrium
ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

# Linear mapping parameters
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# Linear mapping function and its derivative
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

x->r linear mapping parameters 'a': [0.11245619 0.11245619 0.17845517]
x->r linear mapping parameters 'b': [1.3358387  1.3358387  1.61042427]


Precompute matrix elements of expansion terms in Hermite basis, i.e., $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\mathcal{H}_j\rangle$, $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, and $\langle\partial_{r_\lambda}\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, for three internal valence coordinates $\lambda =1..3$.

In [ ]:
no_bas = [20] * ncoo
npoints = [80] * ncoo

prim_me = []
prim_dme = []
prim_d2me = []

for icoo in range(ncoo):
    x, w = hermgauss(npoints[icoo])
    w /= np.exp(-(x**2))
    r = x_to_r_map(x, icoo)
    dr = r - r0[icoo]
    r_pow = dr[:, None] ** powers[icoo][None, :]
    psi = hermite(x, np.arange(no_bas[icoo] + 1))
    dpsi = (
        hermite_deriv(x, np.arange(no_bas[icoo] + 1)) / jac_x_to_r_map(x, icoo)[:, None]
    )
    me = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, psi, w)
    dme = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, dpsi, w)
    d2me = jnp.einsum("gi,gt,gj,g->tij", dpsi, r_pow, dpsi, w)
    prim_me.append(me)
    prim_dme.append(dme)
    prim_d2me.append(d2me)

Solve contracted problems ...

In [ ]:
contr_vec = []

for icoo in range(ncoo):

    bas_ind = [
        np.arange(no_bas[icoo] + 1) if icoo_ == icoo else np.arange(1)
        for icoo_ in range(ncoo)
    ]
    bas_ind, bas_mind = next(generate_prod_ind(bas_ind))

    me = np.prod(
        [
            prim_me[icoo_][
                np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
            ]
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    d2me = np.prod(
        [
            (
                prim_d2me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
                if icoo_ == icoo
                else prim_me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
            )
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    vme = me.T @ poten_coefs
    gme = d2me.T @ Gmat_coefs[:, icoo, icoo]
    hme = 0.5 * gme + vme
    e, v = np.linalg.eigh(hme)
    print(icoo, e[0], e - e[0])
    contr_vec.append(v)

0 2361.7950570907747 [     0.           2627.83310096   5165.68583661   7629.77903368
  10056.05687722  12496.84714452  15001.69278415  17602.2923534
  20313.71106937  23141.21204003  26085.54001494  29158.08835826
  32412.02243726  35975.96051399  40073.32846806  44265.74056481
  49520.08920755  60164.42659295  80827.07835753 118195.6035581
 192185.37003263]
1 2361.7950571375327 [     0.           2627.83310096   5165.6858366    7629.77903364
  10056.05687712  12496.84714429  15001.69278373  17602.29235276
  20313.71106847  23141.21203885  26085.54001347  29158.0883565
  32412.02243523  35975.96051167  40073.32846549  44265.74056187
  49520.08920387  60164.42658833  80827.07835174 118195.60355008
 192185.37001902]
2 2026.5007139178244 [    0.          1214.1828581   2422.52098473  3624.57606586
  4819.78085719  6007.22797727  7185.24499818  8350.44883354
  9494.45913345 10573.94144988 11195.07551955 11884.93586704
 12954.76498972 14079.89600022 15258.65700998 16485.15096834
 17803.263

Transform primitive matrix elements to the contracted basis

In [ ]:
contr_me = [contr_vec[icoo].T @ prim_me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_dme = [contr_vec[icoo].T @ prim_dme[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_d2me = [contr_vec[icoo].T @ prim_d2me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]

Final solution 

In [ ]:
pmax = 12
poly_coefs = np.array([2, 2, 1])

bas_ind = [np.arange(nbas[icoo] + 1) for icoo in range(ncoo)]
bas_ind, bas_mind = next(
    generate_prod_ind(bas_ind, select=lambda ind: np.sum(ind * poly_coefs) <= pmax)
)
print("polyad contraction number:", pmax)
print("total number of basis products:", len(bas_ind))

me = np.prod(
    [
        contr_me[icoo_][
            np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
        ]
        for icoo_ in range(ncoo)
    ],
    axis=0,
)

vme = me.T @ poten_coefs

gme = 0
fac = {}
for icoo in range(ncoo):
    for jcoo in range(ncoo):
        fac[icoo]=1
        fac[jcoo]=-1

        if icoo != jcoo:
            dme = np.prod(
                [
                    (
                        fac[icoo_]*contr_dme[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo or icoo_ == jcoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        else:
            dme = np.prod(
                [
                    (
                        contr_d2me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        gme += dme.T @ Gmat_coefs[:, icoo, jcoo]


hme = 0.5 * gme + vme
e, v = np.linalg.eigh(hme)
print(e[0], e - e[0])

polyad contraction number: 12
total number of basis products: 140
3300.812959773243 [    0.          1182.65669984  2354.03526785  2614.96653302
  2629.17452148  3513.71874995  3779.97399184  3790.16808876
  4661.0741687   4933.63712603  4940.11760148  5151.11526201
  5153.00665699  5245.23574808  5795.01872261  6075.47115904
  6078.41499941  6295.15870418  6295.69441616  6387.76999692
  6913.57306331  7204.21313212  7204.71082564  7427.33553857
  7427.79204805  7519.60678044  7606.54135726  7606.66902216
  7761.4567233   7788.65428409  8012.78971612  8316.39783804
  8320.2917218   8547.43437398  8548.66373341  8640.21400403
  8729.97959813  8730.29329308  8889.42450278  8907.85856274
  9083.6583926   9413.78034837  9422.0896032   9656.11413126
  9657.80572732  9749.45212026  9843.1829964   9843.59636478
  9935.44634628 10006.52687312 10015.09043862 10015.48501902
 10017.20862814 10227.43644892 10228.68210183 10314.73095526
 10374.98789289 10494.16396706 10533.12682853 10754.45848638
 